In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
import sys
#sys.path.append('/home/diego/Trabajo/Proyectos/Pynterpred_devel/')
sys.path.append('/home/diego/Projects/Pynterpred_devel/')
import pynterpred as pnt
import numpy as np

In [3]:
receptor = pnt.Receptor('../testsystems/Barnase-Barstar/pdbs/Barnase.pdb','amber14-all.xml',pH=7.0)
ligand   = pnt.Ligand('../testsystems/Barnase-Barstar/pdbs/Barstar.pdb','amber14-all.xml',pH=7.0)

In [4]:
context  = pnt.MMContext(receptor,ligand)

In [5]:
region   = pnt.Region(receptor, ligand, delta_x=1.2, nside=3)

In [6]:
docking  = pnt.Docker(context,region)

In [9]:
docking.evaluation()

100%|██████████| 108/108 [00:27<00:00,  3.92it/s]

3369


In [10]:
docking.region.net.number_of_nodes()

8619